In [78]:
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot
import plotly.plotly as py 

init_notebook_mode(connected=True)

In [3]:
def xy_sample(data, size):
    data_len = data.shape[0]
    if data_len > size:
        sub_val = np.random.choice(np.arange(data.shape[0]), size, replace=False)
    else:
        sub_val = np.arange(data.shape[0])
    
    return sub_val

In [19]:
def mean_absolute_percentage_error(y_true, y_pred): 
    #y_true, y_pred = check_array(y_true, y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mean_squared_percentage_error(y_true, y_pred): 
    #y_true, y_pred = check_array(y_true, y_pred)
    return np.mean((y_true - y_pred)**2 / y_true) * 100

def get_regression_metrics(rf, train, y_train, valid, y_valid):
    #rf.fit(train, y_train)
    y_pred = rf.predict(valid)
    
    validation = {}
    validation["msle"] = mean_squared_log_error(y_valid, y_pred)
    validation["mae"] = mean_absolute_error(y_valid, y_pred)
    validation["mse"] = mean_squared_error(y_valid, y_pred)
    validation["mape"] = mean_absolute_percentage_error(y_valid, y_pred)
    validation["r2"] = r2_score(y_valid, y_pred)
    validation["rmsle"] = np.sqrt(mean_squared_log_error(y_valid, y_pred))
    validation["rmse"] = np.sqrt(mean_squared_error(y_valid, y_pred))
    validation["preds"] = y_pred
    #validation["rmspe"] = mean_squared_percentage_error(y_valid, y_pred)
    
    y_pred_train = rf.predict(train)
    training = {}
    training["msle"] = mean_squared_log_error(y_train, y_pred_train)
    training["mae"] = mean_absolute_error(y_train, y_pred_train)
    training["mse"] = mean_squared_error(y_train, y_pred_train)
    training["mape"] = mean_absolute_percentage_error(y_train, y_pred_train)
    training["r2"] = r2_score(y_train, y_pred_train)
    training["rmsle"] = np.sqrt(mean_squared_log_error(y_train, y_pred_train))
    training["rmse"] = np.sqrt(mean_squared_error(y_train, y_pred_train))
    training["preds"] = y_pred_train
    
    return validation, training

########################
# Model + Data Loading #
########################
rf = joblib.load('./data/rf_trained.pkl')
train = pd.read_feather('./data/x_train.feather')
valid = pd.read_feather('./data/x_valid.feather')
y_train = pd.read_csv('./data/y_train.csv',header=None).iloc[:,0]
y_valid = pd.read_csv('./data/y_valid.csv',header=None).iloc[:,0]
print('Data Loaded!')

valid_metrics, train_metrics = get_regression_metrics(rf, train, y_train, valid, y_valid)
df_test = pd.DataFrame(valid_metrics).head(1).drop(["preds"], axis = 1).apply(lambda x:round(x, 4))
print('Metrics Loaded!')

Data Loaded!
Metrics Loaded!


In [25]:
valid_metrics['preds']

IndexError: index 191874 is out of bounds for axis 1 with size 82540

In [29]:
x_samp = xy_sample(valid, 3)

In [30]:
valid_metrics['preds'][x_samp]

array([35695.40144665, 17489.42232017, 28185.04059614])

In [31]:
y_valid.loc[x_samp]

77042    32000.0
30499    19000.0
23508    26000.0
Name: 0, dtype: float64

In [3]:
data = pd.read_csv('./data/TrainAndValid.csv')

/Users/imarobit-mac/anaconda2/envs/fastai/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [132]:
rf = joblib.load('./data/rfc_trained.pkl')
train = pd.read_feather('./data/xc_train.feather')
valid = pd.read_feather('./data/xc_valid.feather')
y_train = pd.read_csv('./data/yc_train.csv',header=None).iloc[:,0]
y_valid = pd.read_csv('./data/yc_valid.csv',header=None).iloc[:,0]

def pred_to_hard(val, arr):
    arr[arr > val] = 1
    arr[arr <= val] = 0
    return arr

def hard_pred_df(data, target, model):
    df = pd.DataFrame(columns=['Break','TP','TN', 'FP', 'FN'])
    preds = model.predict_proba(data)[:,1]
    for i,v in enumerate(np.arange(0,1.001,.001)):
        hard = pred_to_hard(v, preds.copy())
        tp_ = np.sum(np.logical_and((hard == 1), (y_valid.values == 1)))
        tn_ = np.sum(np.logical_and((hard == 0), (y_valid.values == 0)))
        fp_ = np.sum(np.logical_and((hard == 1), (y_valid.values == 0)))
        fn_ = np.sum(np.logical_and((hard == 0), (y_valid.values == 1)))
        df.loc[i] = {
                     'Break':v,
                     'TP':tp_,
                     'TN':tn_,
                     'FP':fp_,
                     'FN':fn_
                    }
    return df

hard_pred_df(valid,y_valid,rf)